In [1]:
from torch.utils.data import DataLoader
from dataset import PathfinderDataset
dataset = PathfinderDataset("/Users/gcodes/Downloads/code/pathfinder_full/baseline")
dataloader = DataLoader(dataset, batch_size=4,shuffle=True, num_workers=4)

In [7]:
import torch
from torch import nn
from hgru import hConvGRUCell, hConvGRU

model = hConvGRU(timesteps=4, filt_size=15)
model = nn.DataParallel(model)

Training with filter size: 15 x 15


In [8]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data['image']
        labels = data['label']

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')